In [93]:
import pandas as pd
from pandas_datareader import data as web
import datetime as dt

In [94]:
# Nome del file (assicurati che sia nella stessa cartella del programma)
file_path = 'Nifty_100_Mid_cleaned_USD.csv'

# Lettura del CSV
# parse_dates=['Date']: Converte automaticamente la colonna stringa in oggetti datetime
# index_col='Date': Imposta la data come indice (molto utile per grafici e analisi temporali)
df_sp = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
df_sp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4792 entries, 2006-01-03 to 2025-12-31
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open         4792 non-null   float64
 1   High         4792 non-null   float64
 2   Low          4792 non-null   float64
 3   Last         4792 non-null   float64
 4   Volume       4792 non-null   float64
 5   Log_Returns  4792 non-null   float64
dtypes: float64(6)
memory usage: 262.1 KB


In [95]:
# Periodo desiderato
start = dt.datetime(2005, 1, 1)
end = dt.datetime(2025, 12, 31)

# Scarica il rendimento giornaliero
tbill3m = web.DataReader('DTB3', 'fred', start, end)

# Pulisci i dati
tbill3m = tbill3m.dropna()

print(tbill3m.tail())
tbill3m.info()

            DTB3
DATE            
2025-12-24  3.60
2025-12-26  3.56
2025-12-29  3.57
2025-12-30  3.55
2025-12-31  3.57
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5253 entries, 2005-01-03 to 2025-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DTB3    5253 non-null   float64
dtypes: float64(1)
memory usage: 82.1 KB


In [96]:
# ---------------------------------------------------------
# MERGE DEI DATI (Aggiunta colonna DTB3 a df_sp)
# ---------------------------------------------------------

df_sp = df_sp.join(tbill3m, how='left')

df_sp['DTB3'] = df_sp['DTB3'].ffill()
df_sp['DTB3'] = df_sp['DTB3'].bfill()

# ---------------------------------------------------------
# VERIFICA
# ---------------------------------------------------------
print("\nInfo aggiornate:")
df_sp.info()

# Controllo rapido se ci sono ancora valori nulli nella colonna DTB3
nulls = df_sp['DTB3'].isnull().sum()
print(f"\nValori mancanti in DTB3 dopo il merge: {nulls}")


Info aggiornate:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4792 entries, 2006-01-03 to 2025-12-31
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open         4792 non-null   float64
 1   High         4792 non-null   float64
 2   Low          4792 non-null   float64
 3   Last         4792 non-null   float64
 4   Volume       4792 non-null   float64
 5   Log_Returns  4792 non-null   float64
 6   DTB3         4792 non-null   float64
dtypes: float64(7)
memory usage: 299.5 KB

Valori mancanti in DTB3 dopo il merge: 0


In [97]:
# ---------------------------------------------------------
# IMPORTAZIONE E MERGE DI MSCI WORLD
# ---------------------------------------------------------

# 1. Caricamento del file CSV
file_path_msci = 'MSCI_World_cleaned.csv'
df_msci = pd.read_csv(file_path_msci, parse_dates=['Date'], index_col='Date')

# Visualizziamo le colonne originali per sicurezza
print("Colonne originali in MSCI World:", df_msci.columns.tolist())

# 2. Rinomina delle colonne come richiesto
# Nota: Assumo che le colonne nel CSV si chiamino 'Last' e 'Log_Returns'.
# Se nel tuo CSV hanno nomi diversi (es. 'Close', 'Returns'), modifica la parte sinistra del dizionario qui sotto.
df_msci = df_msci.rename(columns={
    'Last': 'MSCI World Last',
    'Log_Returns': 'MSCI World Returns'
})

# Selezioniamo solo le due colonne rinominate per evitare di importare colonne extra (es. Volume, Open, ecc.)
df_msci = df_msci[['MSCI World Last', 'MSCI World Returns']]

# 3. Merge con il dataframe principale (df_sp)
# Usiamo 'left' join per mantenere la struttura temporale dell'S&P 400
df_sp = df_sp.join(df_msci, how='left')

# 4. Pulizia opzionale dei dati mancanti (Alignment)
# Se l'S&P era aperto ma l'MSCI chiuso, riempiamo l'ultimo prezzo (ffill)
df_sp['MSCI World Last'] = df_sp['MSCI World Last'].ffill()
# Per i ritorni, se manca il dato (mercato chiuso), assumiamo 0 variazione invece di copiare il ritorno precedente
df_sp['MSCI World Returns'] = df_sp['MSCI World Returns'].fillna(0)

# ---------------------------------------------------------
# VERIFICA FINALE
# ---------------------------------------------------------
print("\n--- DATAFRAME COMPLETATO (S&P 400 + T-Bill + MSCI World) ---")
df_sp.info()


Colonne originali in MSCI World: ['Last', 'Log_Returns']

--- DATAFRAME COMPLETATO (S&P 400 + T-Bill + MSCI World) ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4792 entries, 2006-01-03 to 2025-12-31
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                4792 non-null   float64
 1   High                4792 non-null   float64
 2   Low                 4792 non-null   float64
 3   Last                4792 non-null   float64
 4   Volume              4792 non-null   float64
 5   Log_Returns         4792 non-null   float64
 6   DTB3                4792 non-null   float64
 7   MSCI World Last     4792 non-null   float64
 8   MSCI World Returns  4792 non-null   float64
dtypes: float64(9)
memory usage: 374.4 KB


In [98]:
df_sp

,Open,High,Low,Last,Volume,Log_Returns,DTB3,MSCI World Last,MSCI World Returns
Date,,,,,,,,,
2006-01-03,90.284314,91.409760,90.284314,91.383122,4.618471e+07,0.015652,4.07,1281.30,0.017470
2006-01-04,92.222967,92.774799,92.133601,92.649687,4.516293e+07,0.013765,4.09,1295.12,0.010728
2006-01-05,92.877585,93.490218,92.280604,93.381778,6.392667e+07,0.007871,4.10,1295.40,0.000216
2006-01-06,93.816306,94.613659,93.436133,94.274980,6.614162e+07,0.009520,4.12,1308.15,0.009794
2006-01-09,95.470037,95.875183,95.403453,95.656246,5.715597e+07,0.014545,4.14,1309.42,0.000970
...,...,...,...,...,...,...,...,...,...
2025-12-24,678.060815,679.767766,672.829695,673.327579,9.364248e+08,-0.007325,3.60,4474.20,0.002289
2025-12-26,672.512173,676.198770,670.620183,671.260677,9.259862e+08,-0.003074,3.56,4473.23,-0.000304
2025-12-29,671.223544,672.309883,666.057465,666.820402,9.748189e+08,-0.006637,3.57,4461.19,-0.002695


In [99]:
# ---------------------------------------------------------
# ESPORTAZIONE IN CSV
# ---------------------------------------------------------

# Definisci il nome del file di output
output_file = 'Nifty100_Ready_Data.csv'

# Esportazione del dataframe
# index=True serve per includere la colonna 'Date' nel file CSV
df_sp.to_csv(output_file, index=True)

print(f"Esportazione completata con successo! Il file è stato salvato come: {output_file}")

Esportazione completata con successo! Il file è stato salvato come: Nifty100_Ready_Data.csv
